In [1]:
import pandas as pd
import sqlalchemy  # Import the module
from sqlalchemy import create_engine  
from time import time
from concurrent.futures import ThreadPoolExecutor
import configparser


In [5]:
config = configparser.ConfigParser()
config.read('config.ini')


username = config['database']['username']
password = config['database']['password']
host = config['database']['host']
port = config['database']['port']
database_name = config['database']['database_name']
dataset_path = config['paths']['dataset_path']
file_name = config['paths']['file_name']


Username: root
Password: root
Host: localhost
Port: 5432
Database Name: ny_taxi
Dataset Path: /home/vivek/de_zoomcamp_2025/dataset/
File Name: yellow_tripdata_2021-01.csv.gz


In [ ]:
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{database}")
df_iter = pd.read_csv(path + file, compression='gzip',parse_dates=['tpep_pickup_datetime' , 'tpep_dropoff_datetime'], iterator=True ,chunksize=100000,low_memory=False)
df = next(df_iter)
df.head(n=0).to_sql(con =engine , name = 'yellow_taxi_data' , if_exists='replace')

In [ ]:


# Define the function to process each chunk
def process_chunk(chunk_count, df_chunk):
    t_start = time()
    df_chunk.to_sql(con=engine, name='yellow_taxi_data', if_exists='append')
    t_end = time()
    print(f'Inserted chunk {chunk_count}, took {t_end - t_start:.3f} seconds')

chunk_count = 1
threads = []

# Initialize ThreadPoolExecutor
abs_start = time()
with ThreadPoolExecutor(max_workers=None) as executor:  # Adjust max_workers based on your resources
    while True:
        try:
            # Submit a chunk for processing to the thread pool
            threads.append(executor.submit(process_chunk, chunk_count, df))
            chunk_count += 1
            df = next(df_iter)
        except StopIteration:
            print("All chunks have been processed.")
            break
# Initialize ThreadPoolExecutor
abs_end = time() - abs_start
print(f"{abs_end:.3f} seconds")

